### EDA on data about amount of people studying for specific fields in YH and amount of people finishing their studies. 


In [1]:
import pandas as pd
import duckdb
import matplotlib.pyplot as plt
import plotly.express as px